In [1]:
import os
import sys
import numpy as np
import pandas as pd
import openmatrix as omx

In [ ]:
os.chdir('../data/TransportationNetworks')
root = os.getcwd()
root

In [ ]:
folders = [x for x in os.listdir(root)[1:] if os.path.isdir(os.path.join(root, x))]
folders

In [4]:
# Function to import OMX matrices
def import_matrix(matfile):
    f = open(matfile, 'r')
    all_rows = f.read()
    blocks = all_rows.split('Origin')[1:]
    matrix = {}
    for k in range(len(blocks)):
        orig = blocks[k].split('\n')
        dests = orig[1:]
        orig=int(orig[0])

        d = [eval('{'+a.replace(';',',').replace(' ','') +'}') for a in dests]
        destinations = {}
        for i in d:
            destinations = {**destinations, **i}
        matrix[orig] = destinations
    zones = max(matrix.keys())
    mat = np.zeros((zones, zones))
    for i in range(zones):
        for j in range(zones):
            mat[i, j] = matrix.get(i+1,{}).get(j+1,0)
            
    return mat

In [ ]:
import_matrix(os.path.join(root, 'SiouxFalls', 'SiouxFalls' + '_trips.tntp'))

In [5]:
# Importing the networks into a Pandas dataframe consists of a single line of code
# but we can also make sure all headers are lower case and without trailing spaces
def import_net(f):
    netfile = os.path.join(root, f, f + '_net.tntp')
    net = pd.read_csv(netfile, skiprows=8, sep='\t')

    trimmed= [s.strip().lower() for s in net.columns]
    net.columns = trimmed

    # And drop the silly first andlast columns
    net.drop(['~', ';'], axis=1, inplace=True)
    return net

In [ ]:
import_net('Braess')

In [9]:
def process_dataset(net_name): 
    mat = import_matrix(os.path.join(root, net_name, net_name + '_trips.tntp'))
    net = import_net(net_name)
    number_of_zones = mat.shape[0]
    number_of_nodes = net[['init_node', 'term_node']].max().max()
    number_of_links = net.shape[0]
    with open(os.path.join(root, net_name,  net_name + '_net.csv'), 'w') as f:
        f.write(f"# NUMBER OF ZONES:{number_of_zones},")
        f.write(f"NUMBER OF NODES:{number_of_nodes},")
        f.write(f"NUMBER OF LINKS:{number_of_links}")
        f.write("\n")

        net.to_csv(f, index=False)
    
    with open(os.path.join(root, net_name,  net_name + '_trips.csv'), 'w') as f:
        f.write(f"# NUMBER OF ZONES:{number_of_zones}")
        f.write("\n") 
        
        np.savetxt(f, mat, delimiter=',', fmt='%d')

In [10]:
process_dataset('Braess')